In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib.parse as urlparse
import re
import random
import webbrowser
import requests
from re import sub
from decimal import Decimal
import numpy as np
import pandas as pd
import urllib

### Extract name list from kaggle data

In [ ]:
df_kaggle = pd.read_csv("../kaggle_data/train.csv",sep=",")

name_of_movie = df_kaggle.original_title

lan_list =  df_kaggle.original_language
en_list = [x for x in lan_list if x!="en"]
en_indx = df_kaggle[df_kaggle.original_language=="en"]["id"]

In [ ]:
movie_num = len(df_kaggle.id)
df_kaggle["first_week_box_office"] = pd.Series([None]*movie_num, index=df_kaggle.index)

### Search the corresponding first-week box office

In [ ]:
def find_box_office(movie_id=None,box=None):
    if movie_id == None:
        return None
    else:
        base_url_movie = "https://www.boxofficemojo.com/movies/"
        param_movie = {"page":"weekend","id":movie_id}
        r_ = requests.get(base_url_movie, params=param_movie)
        url_ = r_.url
        html_ = urlopen(url_).read()
        soup_ = BeautifulSoup(html_,'lxml')
        sub_ = soup_.find_all('font',{'size':re.compile('2'), 'face':None})
        # title_ = soup_.find_all("font",{"face":re.compile("Verdana"),"size":re.compile("[4-6]")})
        # title_ = title_[0].get_text()
        for k in range(len(sub_)):
            if ('$' in sub_[k].get_text()):
                box = sub_[k].get_text()
                return Decimal(sub(r'[^\d.]', '', box))
        return box

In [ ]:
def find_webID(movie_name,url_list,webID=None):
    if url_list is None:
        return None
    else:
        for i in range(len(url_list)):
            if(movie_name in url_list[i].get_text() or url_list[i].get_text() in movie_name):
                webID = urlparse.parse_qs(urlparse.urlparse(url_list[i]["href"]).query)
                return webID["id"]
        return None

In [ ]:
def search_movie(movie_name):
    url_search = "https://www.boxofficemojo.com/search/"
    param = {'q': movie_name}
    r_search = requests.get(url_search,params=param)
    url_search = r_search.url
    try:
        html_search = urlopen(url_search)
    except:
        return None
    html_search = urlopen(url_search)
    soup_search = BeautifulSoup(html_search,"lxml")
    sub_url_search = soup_search.find_all("a",{"href":re.compile("/movies/\?id")})
    return sub_url_search

In [ ]:
url_search = "https://www.boxofficemojo.com/search/"
# name_of_movie = ["Insight","Control Room","A Mighty Wind"]

counter = 0 

for i in en_indx :
    sub_url = search_movie(movie_name=name_of_movie[i-1])
    
    webID = find_webID(name_of_movie[i-1],sub_url)
    
    # par_search_id = urlparse.parse_qs(urlparse.urlparse(sub_url_search[1]["href"]).query)
    
    box_office = find_box_office(webID)
    df_kaggle.first_week_box_office[i-1] = box_office
    #print(i," <",name_of_movie[i-1],">"," the first week box office is: ",box_office)
    
   
    if counter%25==0:
        print("###### Searching for the box office ####### Completed: {:.2%}".format(counter/len(en_indx),".2%"))
    
    counter+=1
    ####### comment this out when establishing the data set #######
    #    if counter==200:
    #        break
    ###############################################################
print("##### Finished #####")

### Export the data

In [ ]:
df_kaggle.to_excel("fwbo_kaggle.xls",columns=df_kaggle.columns,index=False)